In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader

from torchvision import transforms

from medmnist import BloodMNIST, INFO

import argparse
import numpy as np
from matplotlib import pyplot as plt
from sklearn.metrics import accuracy_score
import time


OSError: [WinError 1114] A dynamic link library (DLL) initialization routine failed. Error loading "c:\Python312\Lib\site-packages\torch\lib\c10.dll" or one of its dependencies.

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
batch_size = 64

# Data Loading

data_flag = 'bloodmnist'
print(data_flag)
info = INFO[data_flag]
print(len(info['label']))
n_classes = len(info['label'])

# Transformations
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[.5], std=[.5])
])

In [ ]:
# --------- Before Training ----------
total_start = time.time()


In [ ]:
def train_epoch(loader, model, criterion, optimizer):
    
    ### YOUR CODE HERE ###
    model.train()
    total_loss = 0.0
    for imgs, labels in loader:
        imgs = imgs.to(device)
        labels = labels.squeeze().long().to(device)

        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()


    return total_loss / len(loader)

In [ ]:
def evaluate(loader, model, apply_softmax=False):
    model.eval()
    preds, targets = [], []

    with torch.no_grad():
        for imgs, labels in loader:
            imgs = imgs.to(device)
            labels = labels.squeeze().long()

            outputs = model(imgs, apply_softmax=apply_softmax)
            preds += outputs.argmax(dim=1).cpu().tolist()
            targets += labels.tolist()

    return accuracy_score(targets, preds)

In [ ]:
def plot(epochs, plottable, ylabel='', name=''):
    plt.clf()
    plt.xlabel('Epoch')
    plt.ylabel(ylabel)
    plt.plot(epochs, plottable)
    plt.savefig('%s.pdf' % (name), bbox_inches='tight')

In [ ]:
class CNNModel(nn.Module):
    def __init__(self, in_channels, num_classes, input_hw: tuple[int, int]):
        super().__init__()
        H, W = input_hw
        self.conv1 = nn.Conv2d(in_channels, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)

        self.relu = nn.ReLU(inplace=True)

        in_features_fc1 = 64 * H * W  # as requested: channels * width * height (from second block)

        # --- Fully connected layers ---
        self.fc1 = nn.Linear(in_features_fc1, 256)
        self.fc2 = nn.Linear(256, num_classes)
        

    def forward(self, x, apply_softmax=False):
        x = self.relu(self.conv1(x))

        # Block 2
        x = self.relu(self.conv2(x))
        x_second = x  # (N, 64, H, W) - used conceptually for the fc input size

        # Block 3
        x = self.relu(self.conv3(x))

        # Flatten using the SECOND block size requirement:
        # We ignore block3's channels for the FC input on purpose because the prompt asks so.
        # So: flatten x_second, not x.
        x = torch.flatten(x_second, start_dim=1)  # (N, 64*H*W)

        # FCs
        x = self.relu(self.fc1(x))
        logits = self.fc2(x)

        if apply_softmax:
            return F.softmax(logits, dim=1)  # probabilities
        return logits  # raw scores (logits)


In [ ]:
train_dataset = BloodMNIST(split='train', transform=transform, download=True, size=28)
val_dataset   = BloodMNIST(split='val',   transform=transform, download=True, size=28)
test_dataset  = BloodMNIST(split='test',  transform=transform, download=True, size=28)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader  = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
model = CNNModel(in_channels=3, num_classes=n_classes, input_hw=(28, 28)).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
epochs = 200
train_losses = []
val_accs = []
test_accs = []
softmax_accs = []
epochs_without_improvement = 0
best_val_acc = 0.0
for epoch in range(epochs):

    epoch_start = time.time()

    train_loss = train_epoch(train_loader, model, criterion, optimizer)
    val_acc = evaluate(val_loader, model)
    print(f"Epoch {epoch+1}/{epochs} | Loss: {train_loss:.4f} | Val Acc: {val_acc:.4f} ")

    train_losses.append(train_loss)
    val_accs.append(val_acc)

    epoch_end = time.time()
    epoch_time = epoch_end - epoch_start

    print(f"Epoch {epoch+1}/{epochs} | "
          f"Loss: {train_loss:.4f} | Val Acc: {val_acc:.4f} | "
          f"Time: {epoch_time:.2f} sec")
    
    # Early Stopping
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        epochs_without_improvement = 0
    else:
        epochs_without_improvement += 1

    if epochs_without_improvement >= 5:
        print("Early stopping triggered.")
        break

#Test Accuracy
test_acc = evaluate(test_loader, model)
print("Test Accuracy:", test_acc)
test_accs.append(test_acc)


#Save the model
torch.save(model.state_dict(), "bloodmnist_cnn.pth")
print("Model saved as bloodmnist_cnn.pth")

# --------- After Training ----------
total_end = time.time()
total_time = total_end - total_start

print(f"\nTotal training time: {total_time/60:.2f} minutes "
      f"({total_time:.2f} seconds)")

In [ ]:
config = "{}".format(str(0.1))

plot(epochs, train_losses, ylabel='Loss', name='CNN-training-loss-{}'.format(config))
plot(epochs, val_accs, ylabel='Accuracy', name='CNN-validation-accuracy-{}'.format(config))
plot(epochs, test_accs, ylabel='Accuracy', name='CNN-test-accuracy-{}'.format(config))